In [11]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Day4") \
    .getOrCreate()

from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

In [6]:
data = [
    (1, "Alice", 5, "P"),
    (2, "Bob", 5, "A"),
    (3, "Charlie", 6, "P"),
    (4, "David", 6, "P"),
    (5, "Eva", 5, "P"),
]

columns = ["student_id", "name", "grade", "attendance_code"]

df = spark.createDataFrame(data, columns)

df.show()

window_spec = Window.partitionBy("grade").orderBy("student_id")

+----------+-------+-----+---------------+
|student_id|   name|grade|attendance_code|
+----------+-------+-----+---------------+
|         1|  Alice|    5|              P|
|         2|    Bob|    5|              A|
|         3|Charlie|    6|              P|
|         4|  David|    6|              P|
|         5|    Eva|    5|              P|
+----------+-------+-----+---------------+



In [7]:
df_with_rownum = df.withColumn(
    "row_number",
    row_number().over(window_spec)
)

df_with_rownum.show()


+----------+-------+-----+---------------+----------+
|student_id|   name|grade|attendance_code|row_number|
+----------+-------+-----+---------------+----------+
|         1|  Alice|    5|              P|         1|
|         2|    Bob|    5|              A|         2|
|         5|    Eva|    5|              P|         3|
|         3|Charlie|    6|              P|         1|
|         4|  David|    6|              P|         2|
+----------+-------+-----+---------------+----------+



In [8]:
df_with_rownum = df.withColumn(
    "row_number",
    row_number().over(window_spec)
)

df_with_rownum.show()


+----------+-------+-----+---------------+----------+
|student_id|   name|grade|attendance_code|row_number|
+----------+-------+-----+---------------+----------+
|         1|  Alice|    5|              P|         1|
|         2|    Bob|    5|              A|         2|
|         5|    Eva|    5|              P|         3|
|         3|Charlie|    6|              P|         1|
|         4|  David|    6|              P|         2|
+----------+-------+-----+---------------+----------+



In [13]:
df_with_rownum.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Window [row_number() windowspecdefinition(grade#2L, student_id#0L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS row_number#35], [grade#2L], [student_id#0L ASC NULLS FIRST]
   +- Sort [grade#2L ASC NULLS FIRST, student_id#0L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(grade#2L, 200), ENSURE_REQUIREMENTS, [plan_id=132]
         +- Scan ExistingRDD[student_id#0L,name#1,grade#2L,attendance_code#3]




In [19]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

window_spec = Window.partitionBy("grade") \
                    .orderBy(col("student_id"))

df_ranked = df.withColumn(
    "rn",
    row_number().over(window_spec)
)

df_ranked.show()


+----------+-------+-----+---------------+---+
|student_id|   name|grade|attendance_code| rn|
+----------+-------+-----+---------------+---+
|         1|  Alice|    5|              P|  1|
|         2|    Bob|    5|              A|  2|
|         5|    Eva|    5|              P|  3|
|         3|Charlie|    6|              P|  1|
|         4|  David|    6|              P|  2|
+----------+-------+-----+---------------+---+



In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

window_spec = Window.partitionBy("grade")

df_with_sum = df_ranked.withColumn(
    "total_rns",
    sum("rn").over(window_spec)
)

df_with_sum.count()


5